## ESADE MIBA Artifitial Intelligence Final Project 
### Team Magic Leap (formerly known as Team WWDC 2017)

#### Team Member: Nancy Xiaowen Jiang, Che-Min Tsai, Lifu Zuo

##### Agenda
1. Define the digital event 
2. Definition of the problem (comment lines will be added later in the following notebooks)
3. Scrap the data
4. Clean the dataset (Including Topic analysis)
5. Identify several types of users using different methods
6. Propose a variable for a selected variable that is easy to explain to a decision-maker
7. Propose a variable for the same selected variable that is as accurate as possible

#### In this notebook, we will cover Part 4

In [1]:
#initial - import some basic libraries that might be useful here
%pylab inline
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', 30)
import numpy as np
import pymongo

Populating the interactive namespace from numpy and matplotlib


In [2]:
#connect to mongoDB
try:
    #use your database name, user and password here:
    #mongodb://<dbuser>:<dbpassword>@<mlab_url>/<database_name>
    with open("credentials1.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
        print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 

db = conn['aifinalproject_1']
collection = db['Magic_Leap']

Connected successfully!!!


In [3]:
#see how one collection element looks like
print(collection.find_one())

{'_id': ObjectId('5a3aba680a41186c500a0776'), 'created_at': 'Wed Dec 20 19:30:22 +0000 2017', 'id': 943564250072657920, 'id_str': '943564250072657920', 'full_text': 'People are freaking out over the @magicleap product that almost no one’s seen yet https://t.co/LrQVipZbDN https://t.co/EvGkj5AM6C', 'truncated': False, 'display_text_range': [0, 105], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'magicleap', 'name': 'Magic Leap', 'id': 106109075, 'id_str': '106109075', 'indices': [33, 43]}], 'urls': [{'url': 'https://t.co/LrQVipZbDN', 'expanded_url': 'https://buff.ly/2oXaF49', 'display_url': 'buff.ly/2oXaF49', 'indices': [82, 105]}], 'media': [{'id': 943564247203811329, 'id_str': '943564247203811329', 'indices': [106, 129], 'media_url': 'http://pbs.twimg.com/media/DRg2rwFXkAEBJ89.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DRg2rwFXkAEBJ89.jpg', 'url': 'https://t.co/EvGkj5AM6C', 'display_url': 'pic.twitter.com/EvGkj5AM6C', 'expanded_url': 'https://

In [4]:
#read the csv and trying to create a table

#read the CSV file part 1
data = pd.read_csv("C:/Users/maximjxw/Desktop/AI Final Project/magicleap.csv",index_col = 0)
df = pd.DataFrame(data)

clean_df = df[['created_at','display_text_range','full_text','favorite_count','retweeted','retweeted_status','retweet_count',]]
clean_df.shape

(5018, 7)

In [5]:
#update the text number
text_number = []
for i in clean_df['display_text_range']:
    text_number.append(i.split(',')[1][:-1])

clean_df['text_number'] = text_number
clean_df = clean_df.drop(['display_text_range'], axis = 1)
clean_df.head(2)

D:\PythonSoftware\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number
0,Wed Dec 20 19:30:22 +0000 2017,People are freaking out over the @magicleap pr...,0,False,NaN,0,105
1,Mon Jan 01 00:16:42 +0000 2018,Google’s Magic Leap Just Revealed An Impressiv...,0,False,NaN,0,73


In [6]:
#get all hashtags
allhashtags = []

collection_total = collection.find()

for col in collection_total:
    for i in col['entities']['hashtags']:
        allhashtags.append(i['text'])

#see how it looks like
print(allhashtags[:10])

['AR', 'AugmentedReality', 'AR', 'AugmentedReality', 'magicleap', 'Cloud', 'Tech', 'HeyHeyHeyTODAY', 'كلنا_عبدالله_بن_زايد', 'RomaTorino']


In [7]:
#get all full text
alltext = []

collection_total = collection.find()

for col in collection_total:
    alltext.append(col['full_text'])

#see how it looks like
print(alltext[:2])

['People are freaking out over the @magicleap product that almost no one’s seen yet https://t.co/LrQVipZbDN https://t.co/EvGkj5AM6C', 'Google’s Magic Leap Just Revealed An Impressive Demo Of Augmented Reality']


In [8]:
#see where are all users from
raw_location = []
location = []
collection_total = collection.find()

for col in collection_total:
    raw_location.append(col['user']['location'])
    if col['user']['location'] != '':
        location.append(col['user']['location'])

print(raw_location[:10])
print(location[:10])

['New York, NY', '', 'Vancouver', 'Baketown California, USA', 'San Francisco, CA', 'Berkeley, CA', 'California USA', 'San Francisco, CA', 'Austin, Texas', '127.0.0.1']
['New York, NY', 'Vancouver', 'Baketown California, USA', 'San Francisco, CA', 'Berkeley, CA', 'California USA', 'San Francisco, CA', 'Austin, Texas', '127.0.0.1', 'Malmö, Sweden']


In [10]:
#merge the location in the dataframe
clean_df['location'] = raw_location
clean_df.head()

,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number,location
0,Wed Dec 20 19:30:22 +0000 2017,People are freaking out over the @magicleap pr...,0,False,NaN,0,105,"New York, NY"
1,Mon Jan 01 00:16:42 +0000 2018,Google’s Magic Leap Just Revealed An Impressiv...,0,False,NaN,0,73,
2,Wed Dec 20 19:30:24 +0000 2017,"RT @magicleap: World, meet Magic Leap One. Mag...",0,False,{'created_at': 'Wed Dec 20 14:01:52 +0000 2017...,1570,131,Vancouver
3,Wed Dec 20 19:30:13 +0000 2017,"After 6 years and $1.9 billion in funding, sec...",0,False,NaN,0,110,"Baketown California, USA"
4,Wed Dec 20 19:30:12 +0000 2017,"Here it is, people, the @magicleap One: https:...",0,False,NaN,0,87,"San Francisco, CA"


In [11]:
#get users' description
raw_description = []
description = []
collection_total = collection.find()

for col in collection_total:
    raw_description.append(col['user']['description'])
    if col['user']['description'] != '':
        description.append(col['user']['description'])

print(raw_description[:2])
print(description[:2])

['Official Twitter feed for the Fast Company business media brand;  inspiring readers to think beyond traditional boundaries & create the future of business.', "Hey,I'm ohd Zulhelmi,1,Alor Sear,Kedah"]
['Official Twitter feed for the Fast Company business media brand;  inspiring readers to think beyond traditional boundaries & create the future of business.', "Hey,I'm ohd Zulhelmi,1,Alor Sear,Kedah"]


In [12]:
#get 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count' & 'statuses_count'
raw_followers_count = []
raw_friends_count = []
raw_listed_count = []
raw_created_at = []
raw_favourites_count = []
raw_statuses_count = []

collection_total = collection.find()

for col in collection_total:
    raw_followers_count.append(col['user']['followers_count'])
    raw_friends_count.append(col['user']['friends_count'])
    raw_listed_count.append(col['user']['listed_count'])
    raw_created_at.append(col['user']['created_at'])
    raw_favourites_count.append(col['user']['favourites_count'])
    raw_statuses_count.append(col['user']['statuses_count'])
    
print(raw_followers_count[:2],
raw_friends_count[:2],
raw_listed_count[:2],
raw_created_at[:2],
raw_favourites_count[:2],
raw_statuses_count[:2])

[2323200, 0] [4026, 128] [44594, 0] ['Wed Mar 28 22:39:21 +0000 2007', 'Sun Dec 03 21:11:06 +0000 2017'] [7814, 1] [174691, 9]


In [13]:
#add users' detail to the dataframe
clean_df['user_followers'] = raw_followers_count
clean_df['user_friends'] = raw_friends_count
clean_df['user_listed'] = raw_listed_count
clean_df['user_created_at'] = raw_created_at
clean_df['user_favourites_count'] = raw_favourites_count
clean_df['user_statuses_number'] = raw_statuses_count

clean_df.head(2)

,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number,location,user_followers,user_friends,user_listed,user_created_at,user_favourites_count,user_statuses_number
0,Wed Dec 20 19:30:22 +0000 2017,People are freaking out over the @magicleap pr...,0,False,NaN,0,105,"New York, NY",2323200,4026,44594,Wed Mar 28 22:39:21 +0000 2007,7814,174691
1,Mon Jan 01 00:16:42 +0000 2018,Google’s Magic Leap Just Revealed An Impressiv...,0,False,NaN,0,73,,0,128,0,Sun Dec 03 21:11:06 +0000 2017,1,9


In [14]:
#check retweet status and make it '1' if it's a retweet, '0' if it's not
retweet_yesorno = []
for text in alltext:
    if 'RT ' in text:
        retweet_yesorno.append(1)
    else:
        retweet_yesorno.append(0)

In [15]:
#add this to the dataframe
clean_df['Is_this_a_retweet'] = retweet_yesorno
clean_df.head()

,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number,location,user_followers,user_friends,user_listed,user_created_at,user_favourites_count,user_statuses_number,Is_this_a_retweet
0,Wed Dec 20 19:30:22 +0000 2017,People are freaking out over the @magicleap pr...,0,False,NaN,0,105,"New York, NY",2323200,4026,44594,Wed Mar 28 22:39:21 +0000 2007,7814,174691,0
1,Mon Jan 01 00:16:42 +0000 2018,Google’s Magic Leap Just Revealed An Impressiv...,0,False,NaN,0,73,,0,128,0,Sun Dec 03 21:11:06 +0000 2017,1,9,0
2,Wed Dec 20 19:30:24 +0000 2017,"RT @magicleap: World, meet Magic Leap One. Mag...",0,False,{'created_at': 'Wed Dec 20 14:01:52 +0000 2017...,1570,131,Vancouver,1007,1231,32,Sun Jun 05 20:55:09 +0000 2011,1567,4990,1
3,Wed Dec 20 19:30:13 +0000 2017,"After 6 years and $1.9 billion in funding, sec...",0,False,NaN,0,110,"Baketown California, USA",15560,4199,162,Sat Nov 20 03:40:07 +0000 2010,26350,32491,0
4,Wed Dec 20 19:30:12 +0000 2017,"Here it is, people, the @magicleap One: https:...",0,False,NaN,0,87,"San Francisco, CA",169,208,133,Thu Jan 21 17:52:00 +0000 2016,77,1993,0
